## Recipe Maker

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from pydantic import BaseModel

In [3]:
class RecipeCreationState(BaseModel):
    """
    Represents the state of a recipe generation workflow.
    """
    ingredients: str
    meal_type: str | None = None
    recipe_steps: str | None = None
    final_recipe: str | None = None

### LLM

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

### Agents

In [5]:
from langchain.schema import HumanMessage

def determine_meal_type(state: RecipeCreationState) -> RecipeCreationState:
    """
    Determines the type of meal based on provided ingredients.
    """
    response = llm.invoke([HumanMessage(content=f"What type of meal can be made with these ingredients? {state.ingredients}")]).content
    state.meal_type = response
    return state

def generate_recipe_steps(state: RecipeCreationState) -> RecipeCreationState:
    """
    Generates cooking steps for the meal type.
    """
    response = llm.invoke([HumanMessage(content=f"Provide cooking steps for making {state.meal_type} with these ingredients: {state.ingredients}")]).content
    state.recipe_steps = response
    return state

def format_final_recipe(state: RecipeCreationState) -> RecipeCreationState:
    """
    Formats the final recipe in a structured manner.
    """
    response = llm.invoke([HumanMessage(content=f"Format a final recipe for {state.meal_type} including ingredients and steps: {state.recipe_steps}")]).content
    state.final_recipe = response
    return state

### Workflow (Nodes)

In [6]:
graph = StateGraph(RecipeCreationState)
graph.add_node("DetermineMealType", determine_meal_type)
graph.add_node("GenerateRecipeSteps", generate_recipe_steps)
graph.add_node("FormatFinalRecipe", format_final_recipe)

### Edges

In [7]:
graph.add_edge("DetermineMealType", "GenerateRecipeSteps")
graph.add_edge("GenerateRecipeSteps", "FormatFinalRecipe")

graph.set_entry_point("DetermineMealType")

### Run the Graph

In [9]:
runnable_graph = graph.compile()

initial_state = RecipeCreationState(ingredients="chicken, garlic, tomatoes, onions, spices")
final_state = RecipeCreationState(**runnable_graph.invoke(initial_state))

print("Generated Recipe:", final_state.final_recipe)


Generated Recipe: Ingredients:
- 1 lb chicken pieces (such as thighs or breasts)
- 1 onion, chopped
- 3 cloves garlic, minced
- 1 can diced tomatoes
- 2 tsp paprika
- 1 tsp cumin
- 1 tsp oregano
- Salt and pepper, to taste
- Olive oil
- Chicken broth

Instructions:
1. Heat a large pot or Dutch oven over medium-high heat. Add a drizzle of olive oil and brown the chicken pieces on all sides, about 5-7 minutes. Remove the chicken and set aside.
2. In the same pot, add chopped onions and garlic. Cook until the onions are translucent, about 5 minutes.
3. Add diced tomatoes to the pot, along with paprika, cumin, and oregano. Stir to combine.
4. Add the browned chicken back to the pot and pour in enough chicken broth to cover the chicken and vegetables. Bring to a boil, then reduce heat and let simmer for about 30 minutes.
5. Taste and adjust seasoning as needed, adding salt and pepper to taste.
6. Serve the chicken and tomato stew over cooked rice or with crusty bread. Enjoy!
